In [4]:
import pandas as pd
import numpy as np
import importlib
from sklearn.preprocessing import StandardScaler

source = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'

iris_df = pd.read_csv(
    filepath_or_buffer=source,
    header=None,
    sep=',')

iris_df.columns=['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']
iris_df.dropna(how="all", inplace=True) # drops the empty line at file-end

iris_values = iris_df.iloc[:,0:4].values
iris_labels = iris_df.iloc[:,4].values
iris_values_std = StandardScaler().fit_transform(iris_values)

iris_values_std_ = list(map(lambda x: np.array(x), iris_values_std))

def labelStr2labelInt(elem):
    if(elem == 'Iris-virginica'):
        return 0
    elif(elem == 'Iris-setosa'):
        return 1
    elif(elem == 'Iris-versicolor'):
        return 2
    else:
        raise ValueError("")

iris_labels_ = list(map(labelStr2labelInt, iris_labels))

iris_df

,sepal_len,sepal_wid,petal_len,petal_wid,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [18]:
import fuzzylearn as fl
from fuzzylearn import fuzzifiers as ff
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import importlib

importlib.reload(fl)

X_train, X_test, y_train, y_test = train_test_split(iris_values_std_, iris_labels, random_state=9098, test_size=0.1)

len_train = len(y_train)

mu_setosa = [1 if y_train[i] == 'Iris-setosa' else 0 for i in range(len_train)]
mu_versicolor = [1 if y_train[i] == 'Iris-versicolor' else 0 for i in range(len_train)]
mu_virginica = [1 if y_train[i] == 'Iris-virginica' else 0 for i in range(len_train)]
model_setosa = fl.FuzzyInductor()
model_versicolor = fl.FuzzyInductor()
model_virginica = fl.FuzzyInductor()
sum(mu_setosa) + sum(mu_virginica) + sum(mu_versicolor) == len_train

gurobi not available


True

In [19]:
model_setosa.fit(X_train, mu_setosa)
model_virginica.fit(X_train, mu_virginica)
model_versicolor.fit(X_train, mu_versicolor)

100%|██████████| 100/100 [01:42<00:00,  1.02s/it]
{}
[0, 0.0076040556, 0.0074859797, 0.0074363365, 0.007427712, 0, 0.007438513, 0.007406278, 0, 0, 0, 0, 0.007442372, 0.0074709766, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0074668, 0, 0, 0, 0.00743944, 0, 0, 0.0074078324, 0, 0, 0.0074381423, 0, 0.00742009, 0, 0.007442372, 0, 0, 0, 0.007480395, 0, 0, 0.00754238, 0.0074168798, 0, 0, 0.007422069, 0, 0, 0, 0, 0, 0, 0, 0.0074405447, 0, 0, 0, 0, 0.007403871, 0.007537529, 0, 0, 0.007418142, 0, 0.0075457306, 0.007442372, 0.007475999, 0, 0, 0.007495836, 0.0076096533, 0, 0.007692647, 0, 0, 0, 0.007463602, 0.0075126784, 0.0074949777, 0, 0, 0.0074570007, 0, 0.007404972, 0, 0.007430319, 0, 0, 0.00743876, 0, 0, 0, 0, 0.0074676503, 0, 0, 0, 0.0075430553, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0075399117, 0, 0, 0.007416207, 0.0075049186, 0.0074171857, 0.0074126977, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0076369364, 0, 0, 0.0074749365, 0, 0.0077022645, 0, 0.0074141636, 0]
Indices SV: [1, 2, 3, 4, 6, 7, 12, 13, 23, 27, 30, 33, 35, 37, 4

FuzzyInductor(c=1,
              fuzzifier=<class 'fuzzylearn.fuzzifiers.ExponentialFuzzifier'>,
              k=GaussianKernel(1), random_state=None, return_profile=False,
              return_vars=False,
              sample_generator=<function FuzzyInductor.fit.<locals>.<lambda> at 0x7f7ad97b3ef0>,
              solve_strategy=(<function solve_optimization_tensorflow at 0x7f7ad9aee710>,
                              {}))

In [20]:
probs = list(zip(map(lambda x: ('Iris-virginica', x), model_virginica.predict(X_test)), map(lambda x: ('Iris-setosa', x), model_setosa.predict(X_test)), map(lambda x: ('Iris-versicolor', x), model_versicolor.predict(X_test))))
results = []
for i in range(len(probs)):
    results.append(sorted(probs[i], key=lambda tup: tup[1])[-1][0])

sum([1 if i[0] == i[1] else 0 for i in list(zip(results, y_test))])/len(y_test)

1.0

In [21]:
probs = list(zip(map(lambda x: ('Iris-virginica', x), model_virginica.predict(X_train)), map(lambda x: ('Iris-setosa', x), model_setosa.predict(X_train)), map(lambda x: ('Iris-versicolor', x), model_versicolor.predict(X_train))))
results = []
for i in range(len(probs)):
    results.append(sorted(probs[i], key=lambda tup: tup[1])[-1][0])

sum([1 if i[0] == i[1] else 0 for i in list(zip(results, y_train))])/len(y_train)

0.9333333333333333

In [26]:
model = cs.Model_PL(c=1, sigma=5, path="./model", model_name="iris-model#7")

train_values, train_labels, test_values, test_labels, train_indices, test_indices = Splitter.split_train_test_dataset(iris_values_std, iris_labels)

NameError: name 'cs' is not defined

In [ ]:
model.fit(train_values, train_labels=list(zip(train_indices, train_labels)), mu=cs.Get_mu(iris_labels))

In [ ]:
model.score(test_values, test_labels, labels_indexed=False)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

kf = KFold(n_splits=3, shuffle=True)

clf = GridSearchCV(model, {'c':[1, 10000], 'sigma': [.1, .25, .225, .5,]}, cv=kf, n_jobs=-1, verbose=10)
clf.fit(train_values, list(zip(train_indices, train_labels)), mu=cs.Get_mu(iris_labels))

In [ ]:
clf.cv_results_

In [ ]:
clf.best_index_

In [ ]:
from tesimodules.ModelSelection import GridSearchCV as gs

clf = gs(model, )